In [1]:
pip install sentencepiece


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/991.5 kB 653.6 kB/s eta 0:00:02
   ---- ----------------------------------- 102.4/991.5 kB 1.2 MB/s eta 0:00:01
   ----- -------------------------------- 143.4/991.5 kB 944.1 kB/s eta 0:00:01
   --------- ------------------------------ 225.3/991.5 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/991.5 kB 1.1 MB/s eta 0:00:01
   ---------- --------------------------- 266.2/991.5 kB 962.4 kB/s eta 0:00:01
   ---------- --------------------------- 266.2/991.5 kB 962.4 kB/s eta 0:00:01
   ------------- ------------------------ 348.2/991.5 kB 864.2 kB/s eta 0:00:01
   --------------------- ------------------ 532.5/991.5 kB 1.2 MB/s eta 0:00:01
   -------------------------- ------------- 645.1/991.5 kB 1.4 MB/s eta 0:00:01
   ---------------------------- ----------- 716.8/991.5 kB 1.2 MB/s eta 0:00:01
   --------------------------------- ------ 839.7

In [3]:
import sentencepiece as spm

# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor(model_file='m.model')


OSError: Not found: "m.model": No such file or directory Error #2

In [1]:
import pandas as pd
import sentencepiece as spm
import torch
from transformers import BertModel

# Load dataset
data = pd.read_excel('Audio_Data.xlsx')

# Train a SentencePiece model
# It requires a large corpus to train; here we assume you've prepared a text file for training.
# Uncomment the following line to train on your data:
# spm.SentencePieceTrainer.train('--input=your_input.txt --model_prefix=m --vocab_size=5000')

# Load the SentencePiece model
sp = spm.SentencePieceProcessor(model_file='m.model')

# Load pre-trained BERT model for embeddings
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get embeddings for a text using SentencePiece
def get_sentencepiece_embeddings(text, max_length=512):
    # Encode text to get subword tokens
    tokens = sp.encode(text, out_type=str)

    # Convert to a format suitable for BERT
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids = token_ids[:max_length]  # truncate if longer than max_length
    padding = [0] * (max_length - len(token_ids))  # padding
    input_ids = token_ids + padding  # add padding

    # Create tensor
    input_tensor = torch.tensor([input_ids])
    
    # Get BERT output embeddings
    with torch.no_grad():
        outputs = model(input_tensor)

    # Extract [CLS] token embedding (representing the entire sequence)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

# Create a new list to hold the features, questions, answers, and strengths
sp_features = []

# Extract embeddings for each row in the dataset
for index, row in data.iterrows():
    question = row['Question']
    answer = row['Answer']

    # Get embeddings for both question and answer
    question_embedding = get_sentencepiece_embeddings(question)
    answer_embedding = get_sentencepiece_embeddings(answer)

    # Concatenate the question and answer embeddings into one feature vector
    combined_embedding = torch.cat((torch.tensor(question_embedding), torch.tensor(answer_embedding)))

    # Append the question, answer, strength, and embedding as a dictionary
    sp_features.append({
        'question': question,
        'answer': answer,
        'strength': row['Strength'],
        'embedding': combined_embedding.numpy()  # SentencePiece embedding as a feature
    })

# Convert the list of dictionaries into a DataFrame
sp_df = pd.DataFrame(sp_features)

# Convert the 'embedding' column (which contains arrays) into a format that can be saved in a CSV
sp_df['embedding'] = sp_df['embedding'].apply(lambda x: ','.join(map(str, x)))

# Save the DataFrame with question, answer, strength, and embeddings to a new CSV
sp_df.to_csv('sentencepiece_features.csv', index=False)

print("SentencePiece feature extraction completed and saved to sentencepiece_features.csv!")


OSError: Not found: "m.model": No such file or directory Error #2

In [ ]:
# # Hyperparameter tuning - using GridSearchCV
# The main goal is to improve the model’s performance on unseen data.
# The right hyperparameters can significantly enhance a model's accuracy, reduce overfitting, and lead to a better generalization.
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Load the BERT features
features = pd.read_csv('sentencepiece_features.csv')

# Convert embeddings and labels
features['embedding'] = features['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=', '))  # Convert string to list
X = np.array(features['embedding'].tolist())
y = features['strength'].values

# Step 2: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Hyperparameter Tuning using Grid Search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit Grid Search
grid_search.fit(X_train.tolist(), y_train)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

# Print the best parameters
print("Best Hyperparameters:", best_params)

# Step 4: Train the final model using the best parameters
final_model = best_rf.fit(X_train.tolist(), y_train)

# Step 5: Evaluate the final model
y_pred = final_model.predict(X_test.tolist())

# Print classification report
print(classification_report(y_test, y_pred))

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


In [ ]:
!pip install catboost xgboost scikit-learn

In [ ]:
# Model Comparison based on the bert model
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

# Load the BERT features
features = pd.read_csv('sentencepiece_features.csv')

# Convert the 'embedding' column from string to a list of floats
def convert_embedding(embedding_str):
    return np.fromstring(embedding_str.strip("[]"), sep=',')

# Apply the conversion to the 'embedding' column
features['embedding'] = features['embedding'].apply(convert_embedding)

# Map strength labels from {-1, 0, 1} to {0, 1, 2}
label_mapping = {-1: 0, 0: 1, 1: 2}
features['mapped_strength'] = features['strength'].map(label_mapping)

# Split the features and labels
X = np.array(features['embedding'].tolist())  # Convert to numpy array for model training
y = features['mapped_strength'].values  # Use the mapped strength column

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of classifiers
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'XGBoost': XGBClassifier(eval_metric='mlogloss', use_label_encoder=False),
    'CatBoost': CatBoostClassifier(silent=True),
    'Naive Bayes': GaussianNB()
}

# Dictionary to store the results
results = []

# Iterate over classifiers and store performance metrics
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Append the results
    results.append([name, accuracy, precision, recall, f1])

# Create a table to show results
headers = ["Classifier", "Accuracy", "Precision", "Recall", "F1-Score"]
print(tabulate(results, headers=headers, tablefmt="grid"))

# If needed: Map predictions back to original labels
inverse_mapping = {0: -1, 1: 0, 2: 1}
y_test_original = np.vectorize(inverse_mapping.get)(y_test)
y_pred_original = np.vectorize(inverse_mapping.get)(y_pred)

# Print classification report for original labels
print(classification_report(y_test_original, y_pred_original))
